In [45]:
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [46]:
masterdata = spark.read.parquet("cleaned_data/Masterdata.parquet")
masterdata.createOrReplaceTempView("masterdata")

inbound_95 = spark.read.parquet("cleaned_data/Inbound_95%.parquet")
inbound_95.createOrReplaceTempView("inbound_95")

inbound_avg = spark.read.parquet("cleaned_data/Inbound_Avg.parquet")
inbound_avg.createOrReplaceTempView("inbound_avg")

inventory = spark.read.parquet("cleaned_data/Inventory.parquet")
inventory.createOrReplaceTempView("inventory")

picking_95 = spark.read.parquet("cleaned_data/Picking_95%.parquet")
picking_95.createOrReplaceTempView("picking_95")

picking_avg = spark.read.parquet("cleaned_data/Picking_Avg.parquet")
picking_avg.createOrReplaceTempView("picking_avg")

In [48]:
spark.sql("""
          SELECT COUNT(DISTINCT SKU) AS DISTINCT_SKUs FROM masterdata
          """).show()

+-------------+
|DISTINCT_SKUs|
+-------------+
|        86174|
+-------------+



In [49]:
spark.sql("""SELECT * FROM masterdata ORDER BY SKU""").show(6)

+--------------+-----------+-----------+----------+-----------+
|           SKU|Weight [kg]|Length [cm]|Width [cm]|Height [cm]|
+--------------+-----------+-----------+----------+-----------+
|20000036909601|       0.11|      38.08|     27.24|       1.24|
|20000208077597|       0.68|      30.41|     18.94|      12.09|
|20000348782059|       0.73|      31.32|     21.83|      12.51|
|20000356661421|        0.7|      33.47|     22.15|      11.92|
|20000374444855|        0.9|      33.86|      22.7|      13.18|
|20000375115161|       0.22|      28.54|      18.0|        3.1|
+--------------+-----------+-----------+----------+-----------+
only showing top 6 rows


In [50]:
spark.sql("""
    SELECT COUNT(*) AS DISTINCT_MODELS
    FROM(
        SELECT SUBSTRING(SKU, 1, 6) as sku_prefix,
            COUNT(DISTINCT SKU) as unique_skus
        FROM masterdata 
        GROUP BY SUBSTRING(SKU, 1, 6)
        ORDER BY sku_prefix
    )
""").show()

+---------------+
|DISTINCT_MODELS|
+---------------+
|          70060|
+---------------+

